<a href="https://colab.research.google.com/github/aranlemaur/Big-Data/blob/main/03_nlp_azure_PII-data-redaction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install azure-ai-textanalytics==5.1.0
!pip install azure-storage-blob

     |████████████████████████████████| 153 kB 9.9 MB/s 
     |████████████████████████████████| 85 kB 2.7 MB/s 
     |████████████████████████████████| 179 kB 55.2 MB/s 
     |████████████████████████████████| 41 kB 816 kB/s 
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.10.0.2
    Uninstalling typing-extensions-3.10.0.2:
      Successfully uninstalled typing-extensions-3.10.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0 requires tf-estimator-nightly==2.8.0.dev2021122109, which is not installed.
arviz 0.11.4 requires typing-extensions<4,>=3.7.4.3, but you have typing-extensions 4.1.1 which is incompatible.
     |████████████████████████████████| 346 kB 8.3 MB/s 
     |████████████████████████████████| 3.6 MB 47.8 MB/s 


In [2]:
key = "dd4328e15d7c41ffb3cdf8f99dad9bc8"
endpoint = "https://nlpwat.cognitiveservices.azure.com/"

from azure.ai.textanalytics import TextAnalyticsClient
from azure.core.credentials import AzureKeyCredential

ta_credential = AzureKeyCredential(key)
client = TextAnalyticsClient(
            endpoint=endpoint, 
            credential=ta_credential)


In [17]:
from azure.storage.blob import BlobClient
# working with PRIVATE data-> in CV example, we took PUBLIC blob. 
#on left menu of STORAGE ACCOUNT-> find connection string below access key
constring = 'DefaultEndpointsProtocol=https;AccountName=nlpblobbo;AccountKey=B/Uuy/XJSvNZRSuh/BSxIftbshRL//d4IRUc6v/wFbZfwi/olHO9vsYRoht1BrI2fLn8+Hb3sMjH+ASti6/64Q==;EndpointSuffix=core.windows.net'
blob = BlobClient.from_connection_string(conn_str=constring, 
                                         container_name="nlpstuff", 
                                         blob_name="pii.txt")
# blob-> written in binary
with open("pii.txt", "wb") as my_blob:
    blob_data = blob.download_blob()
    blob_data.readinto(my_blob)

In [18]:
# text-> read as text
mydata = open("pii.txt", "rt").read()
mydata

'My name is Pikachu Pandey. My phone number is 991-111-1111.\n\nI live at 221 Baker Street, Ghost Lake, Screamville. \n'

In [19]:
mydata = mydata.replace("\n","")
sentokens = mydata.split('.')
print(sentokens)

['My name is Pikachu Pandey', ' My phone number is 991-111-1111', 'I live at 221 Baker Street, Ghost Lake, Screamville', ' ']


In [20]:


# Example method for detecting sensitive information (PII) from text 
def pii_recognition_example(client, mydata):
    documents = mydata
    response = client.recognize_pii_entities(documents, language="en")
    result = [doc for doc in response if not doc.is_error]
    for doc in result:
        print("Redacted Text: {}".format(doc.redacted_text))
        for entity in doc.entities:
            print("Entity: {}".format(entity.text))
            print("\tCategory: {}".format(entity.category))
            print("\tConfidence Score: {}".format(entity.confidence_score))
            print("\tOffset: {}".format(entity.offset))
            print("\tLength: {}".format(entity.length))


In [21]:
pii_recognition_example(client, sentokens)

Redacted Text: My name is **************
Entity: Pikachu Pandey
	Category: Person
	Confidence Score: 0.94
	Offset: 11
	Length: 14
Redacted Text:  My phone number is ************
Entity: 991-111-1111
	Category: PhoneNumber
	Confidence Score: 0.8
	Offset: 20
	Length: 12
Redacted Text: I live at *****************************************
Entity: 221 Baker Street, Ghost Lake, Screamville
	Category: Address
	Confidence Score: 0.84
	Offset: 10
	Length: 41
